## [Pump it Up: Data Mining the Water Table](https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/)

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [3]:
train = pd.read_csv('data/DrivenData/train.csv')
train_targets = pd.read_csv('data/DrivenData/train_targets.csv')
test = pd.read_csv('data/DrivenData/test.csv')
submission = pd.read_csv('data/DrivenData/SubmissionFormat.csv')

In [5]:
pd.set_option('display.max_columns', 500)

In [8]:
train.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [14]:
import re
def add_datepart(df, fldname, drop=True, time=False):
    fld = df[fldname]
    if not np.issubdtype(fld.dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)

In [15]:
add_datepart(train, 'date_recorded')

In [16]:
train.head()

,id,amount_tsh,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,date_recordedYear,date_recordedMonth,date_recordedWeek,date_recordedDay,date_recordedDayofweek,date_recordedDayofyear,date_recordedIs_month_end,date_recordedIs_month_start,date_recordedIs_quarter_end,date_recordedIs_quarter_start,date_recordedIs_year_end,date_recordedIs_year_start,date_recordedElapsed
0,69572,6000.0,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,2011,3,11,14,0,73,False,False,False,False,False,False,1300060800
1,8776,0.0,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,2013,3,10,6,2,65,False,False,False,False,False,False,1362528000
2,34310,25.0,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,2013,2,9,25,0,56,False,False,False,False,False,False,1361750400
3,67743,0.0,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,2013,1,5,28,0,28,False,False,False,False,False,False,1359331200
4,19728,0.0,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,2011,7,28,13,2,194,False,False,False,False,False,False,1310515200


In [17]:
id_column = 'id'

In [29]:
num_columns = [col for col in train._get_numeric_data().columns \
               if col not in id_column and \
               not col.endswith('code')]

In [30]:
train[num_columns].head()

,amount_tsh,gps_height,longitude,latitude,num_private,population,construction_year,date_recordedYear,date_recordedMonth,date_recordedWeek,date_recordedDay,date_recordedDayofweek,date_recordedDayofyear,date_recordedIs_month_end,date_recordedIs_month_start,date_recordedIs_quarter_end,date_recordedIs_quarter_start,date_recordedIs_year_end,date_recordedIs_year_start,date_recordedElapsed
0,6000.0,1390,34.938093,-9.856322,0,109,1999,2011,3,11,14,0,73,False,False,False,False,False,False,1300060800
1,0.0,1399,34.698766,-2.147466,0,280,2010,2013,3,10,6,2,65,False,False,False,False,False,False,1362528000
2,25.0,686,37.460664,-3.821329,0,250,2009,2013,2,9,25,0,56,False,False,False,False,False,False,1361750400
3,0.0,263,38.486161,-11.155298,0,58,1986,2013,1,5,28,0,28,False,False,False,False,False,False,1359331200
4,0.0,0,31.130847,-1.825359,0,0,0,2011,7,28,13,2,194,False,False,False,False,False,False,1310515200


In [32]:
cat_columns = [col for col in train.columns \
               if col not in num_columns and \
                  col != id_column]

In [34]:
for col in cat_columns:
    train[col] = train[col].astype('category')

In [18]:
from lightgbm import LGBMClassifier

In [19]:
lgbm = LGBMClassifier()

In [35]:
lgbm.fit(train.drop(id_column, axis=1),
         train_targets.status_group)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.1, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
        n_jobs=-1, num_leaves=31, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=1)

## Submission

In [36]:
add_datepart(test, 'date_recorded')

In [37]:
for col in cat_columns:
    test[col] = test[col].astype('category')

In [41]:
predicted = lgbm.predict(test.drop(id_column, axis=1))

In [43]:
submission['status_group'] = predicted

In [44]:
submission.head()

,id,status_group
0,50785,non functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional


In [46]:
submission.to_csv('LGBM_baseline.csv', index=False)

## Cross validation

In [58]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss, classification_report, confusion_matrix
from sklearn.model_selection import StratifiedKFold
import time

In [49]:
kfolds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [80]:
results = pd.DataFrame(columns=['dataset', 'model', 'params', 
                                'valid_logloss', 'valid_logloss_std',
                                'train_logloss', 'train_logloss_std',
                                'valid_accuracy', 'valid_accuracy_std',
                                'train_accuracy', 'train_accuracy_std',
                                'logloss_diff', 'time'])

In [86]:
def run_model(model, X, y, X_test, dataset_desc='', params_desc=''):
    model_name = str(model.__class__).split('.')[-1].replace('>','').replace("'",'')
    print(model_name)
    logloss = []
    train_logloss = []
    accuracy = []
    train_accuracy = []
    
    y_test = np.zeros((len(X_test), int(y.nunique()))) # global
    start = time.time()
    for train_idx, test_idx in kfolds.split(X, y):
        model.fit(X.iloc[train_idx,:], y[train_idx], 
                  eval_set=[(X.iloc[test_idx], y.iloc[test_idx])], verbose=False,
#                   eval_metric='multi_logloss', #(lightgbm multi_logloss, xgboost mlogloss)
                  early_stopping_rounds=30 #(lightgbm, xgboost)                  
#                   use_best_model=True,  #(catboost),
#                   cat_features=cat_indices,  #(catboost) global!,
#                   plot=True #(catboost)
                 )
        
        y_ = model.predict_proba(X.iloc[test_idx]) 
        logloss += [log_loss(y.iloc[test_idx], y_)]
        y_ = model.predict_proba(X.iloc[train_idx]) 
        train_logloss += [log_loss(y.iloc[train_idx], y_)]        
        
        y_val = model.predict(X.iloc[test_idx])
        accuracy += [accuracy_score(y.iloc[test_idx], y_val)]
        y_train = model.predict(X.iloc[train_idx])
        train_accuracy += [accuracy_score(y.iloc[train_idx], y_train)]
        
        print(f'Valid Acc: {accuracy[-1]:.6f} | Train Acc: {train_accuracy[-1]:.6f}')
        print(classification_report(y.iloc[test_idx], y_val))
        print(confusion_matrix(y.iloc[test_idx], y_val))        
        
        y_ = model.predict_proba(X_test) 
        y_test += y_
        
        print(f'Valid LL: {logloss[-1]:.6f} | Train LL: {train_logloss[-1]:.6f}')
        print()
        
    y_test /= 5 
    end = time.time()
    results.loc[len(results)] = [dataset_desc, model_name, params_desc,
                                 np.mean(logloss), np.std(logloss),
                                 np.mean(train_logloss), np.std(train_logloss),                                 
                                 np.mean(accuracy), np.std(accuracy),
                                 np.mean(train_accuracy), np.std(train_accuracy),
                                 np.mean(logloss) - np.mean(train_logloss),
                                 round(end - start, 2)]
    print(f'logloss: {np.mean(logloss)}, train_logloss: {np.mean(train_logloss)}')
    return y_test, model

In [82]:
target_column = 'status_group'

In [83]:
y_test, model = run_model(LGBMClassifier(silent=True),
                   train.drop(id_column, axis=1),
                   train_targets[target_column], 
                   test.drop(id_column, axis=1),
                   'baseline', 'no-params')

LGBMClassifier
Valid Acc: 0.797323 | Train Acc: 0.856878
                         precision    recall  f1-score   support

             functional       0.79      0.89      0.84      6452
functional needs repair       0.57      0.32      0.41       864
         non functional       0.83      0.76      0.79      4565

            avg / total       0.79      0.80      0.79     11881

[[5726  137  589]
 [ 463  273  128]
 [1025   66 3474]]
Valid LL: 0.505863 | Train LL: 0.359780

Valid Acc: 0.801364 | Train Acc: 0.856605
                         precision    recall  f1-score   support

             functional       0.80      0.89      0.84      6452
functional needs repair       0.60      0.33      0.43       864
         non functional       0.83      0.77      0.80      4565

            avg / total       0.80      0.80      0.79     11881

[[5719  119  614]
 [ 452  287  125]
 [ 978   72 3515]]
Valid LL: 0.498593 | Train LL: 0.360346

Valid Acc: 0.800084 | Train Acc: 0.855156
           

In [84]:
results

,dataset,model,params,valid_logloss,valid_logloss_std,train_logloss,train_logloss_std,valid_accuracy,valid_accuracy_std,train_accuracy,train_accuracy_std,logloss_diff,time
0,baseline,LGBMClassifier,no-params,0.497902,0.005126,0.360922,0.000735,0.800068,0.002623,0.856149,0.000838,0.13698,19.52


In [98]:
y_test, model = run_model(LGBMClassifier(
                        num_estimators=1000,
                        max_depth=30,
                        reg_alpha=1,
                        reg_lambda=1
                    ),
                   train.drop(id_column, axis=1),
                   train_targets[target_column], 
                   test.drop(id_column, axis=1),
                   'baseline', 'no-params')

LGBMClassifier
Valid Acc: 0.794967 | Train Acc: 0.853911
                         precision    recall  f1-score   support

             functional       0.79      0.89      0.84      6452
functional needs repair       0.57      0.32      0.41       864
         non functional       0.83      0.76      0.79      4565

            avg / total       0.79      0.79      0.79     11881

[[5716  136  600]
 [ 467  273  124]
 [1041   68 3456]]
Valid LL: 0.505503 | Train LL: 0.363758

Valid Acc: 0.802710 | Train Acc: 0.856037
                         precision    recall  f1-score   support

             functional       0.80      0.89      0.84      6452
functional needs repair       0.60      0.34      0.43       864
         non functional       0.83      0.77      0.80      4565

            avg / total       0.80      0.80      0.80     11881

[[5724  115  613]
 [ 453  292  119]
 [ 968   76 3521]]
Valid LL: 0.499159 | Train LL: 0.362303

Valid Acc: 0.799411 | Train Acc: 0.854167
           

In [99]:
results

,dataset,model,params,valid_logloss,valid_logloss_std,train_logloss,train_logloss_std,valid_accuracy,valid_accuracy_std,train_accuracy,train_accuracy_std,logloss_diff,time
0,baseline,LGBMClassifier,no-params,0.497902,0.005126,0.360922,0.000735,0.800068,0.002623,0.856149,0.000838,0.136980,19.52
1,baseline,LGBMClassifier,no-params,0.497890,0.005107,0.362002,0.001654,0.799899,0.002814,0.855619,0.001026,0.135888,19.68
2,baseline,LGBMClassifier,no-params,0.712718,0.002089,0.693955,0.000453,0.771297,0.003112,0.790244,0.001293,0.018763,16.95
3,baseline,LGBMClassifier,no-params,0.712078,0.002098,0.693198,0.000501,0.771229,0.003325,0.790303,0.001288,0.018880,17.56
4,baseline,LGBMClassifier,no-params,0.497890,0.005107,0.362002,0.001654,0.799899,0.002814,0.855619,0.001026,0.135888,19.46
5,baseline,LGBMClassifier,no-params,0.497832,0.005015,0.363743,0.000779,0.799529,0.002820,0.854861,0.000935,0.134088,19.57
